**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
from keras import backend as K
import numpy as np
import io
import base64
from IPython.display import HTML
#import skvideo
#skvideo.setFFmpegPath("/usr/local/bin")
import skvideo.io
import cv2
import json

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Dropout, Flatten
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

```epsilon``` is essential because it will be the exploration rate. For instance, if you have 5 boxes in front of you, you will pick one and earn +5 (which you are trying to maximise) thus every time after that you will pick this box as it goes in the right way. However, there may be a +50 in another box which you wouldn't pick in this case. Hence the necessity of ```epsilon```. In our case, we will see later the great importance of this parameter in the DQN algorithm to get out of local minima !

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature 
        # this will impact the number of positive and negative rewards in the environment

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2, :] = -1
        self.position[:, 0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2,:] = -1
        self.position[:,:2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

At each time step, we need to describe the state in order to take the best possible action.

> ```position``` recalls the very position of the rat at time t (not the visited ones).

> ```board``` recalls the possible reward in each location on the board : either a cheese (+0.5) or poison (-1). Once position has been visited by the rat, it's value on the board is set to zero. On the one hand, this obliges the rat not to oscillate between only two neighbors containing cheese. On the other hand, the rat is able to go back on it's path if he faces a wall of poison for instance. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [95]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1,epochs+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < -0.1:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/e))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/14.0. Average score (-2.5)
Win/lose count 5.5/9.0. Average score (-3.0)
Win/lose count 8.0/13.0. Average score (-3.6666666666666665)
Win/lose count 9.0/8.0. Average score (-2.5)
Win/lose count 8.5/13.0. Average score (-2.9)
Win/lose count 13.5/13.0. Average score (-2.3333333333333335)
Win/lose count 9.5/9.0. Average score (-1.9285714285714286)
Win/lose count 10.5/11.0. Average score (-1.75)
Win/lose count 10.5/12.0. Average score (-1.7222222222222223)
Win/lose count 12.0/19.0. Average score (-2.25)
Final score: -2.25


> A random agent is not great and performs really badly... Let's head to a more suitable approach when actually learning something by reinforcement.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy:

\begin{align}
Q^{\pi}(s,a)&=E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = E[ r(s,a) + \sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] \\
& = r(s,a) + \gamma E[ \sum_{k=0}^{\infty} \gamma^k r_{t+1+k}|s_{t} = s, a_{t} = a]
\end{align}

And since $E[E[X|Y]|Z] = E[X|Z]$ when $Z \subset Y$ which is the case here since the $\sigma$-algebras associated to each time step build a filtration (knowledge at time t is included in knowledge at time t+1):

\begin{align}
Q^{\pi}(s,a)&=r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] \\
& =  r(s,a) + \gamma \sum_{(s',a')}p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
& =  \sum_{(s',a')} p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) r(s,a) + \gamma \sum_{(s',a')}p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a )E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']\\
&=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{align}

---

Then, for the optimal policy $\pi^*$ (we assume its existence):

We define $Q^{*}(s,a)=\max_{\pi}Q^{\pi}(s,a)$ which in our case, the optimal Q function at (s,a) is the maximum over the next possible actions which are chosen by the optimal policy. *NB : the policy chooses the actions so the max over the policy will impact the choice of a.* 

Then,
\begin{align}
Q^{*}(s,a)&=\max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]\\
&=\max_{\pi}\left[\sum_{(s',a')}p(s',a'|s,a)*r(s,a)+p(s',a'|s,a)*\gamma Q^{\pi}(s',a')\right]\\
&=\max_{\pi}\left[r(s,a) + \gamma \sum_{(s',a')}p(s',a'|s,a)*Q^{\pi}(s',a')\right]\\
&=r(s,a) + \gamma \max_{\pi}\left[\sum_{(s',a')}\pi (a'|s')p(s'|s,a)*Q^{\pi}(s',a')\right]\\
&=r(s,a) + \gamma \sum_{s'}p(s'|s,a) \max_{\pi}\left[\sum_{a'}\pi (a'|s')*Q^{\pi}(s',a')\right]\\
&=E_{(s')\sim p(.|s,a)}\left[r(s,a) + \gamma \max_{\pi}[\sum_{a'}\pi (a'|s')*Q^{\pi}(s',a')]\right]
\end{align}
Here, taking the maximum over the policy boils down to considering a policy which will be a one-hot vector choosing the best action to take at this certain state ! Thus, $\max_{\pi}\left[\sum_{a'}\pi (a'|s')*Q^{\pi}(s',a')\right]=\max_{a'}\max_{\pi}Q^{\pi}(s',a')=\max_{a'}Q^{*}(s',a')$

Which leads finally to :

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

---

Finally, we deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

Indeed, it allows to enforce the previous equality. If our neural network perfectly converge this equality must be true. Using this loss function we try to enforce our learned Q-value to verify the Bellman equation.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory)>100:
            self.memory = self.memory[-100:]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [71]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.8 # previous : 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pred_rwd_per_action = self.model.predict(s.reshape(1,5,5,self.n_state))
        return np.argmax(pred_rwd_per_action)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            sample = self.memory.random_access()
            input_states[i,:,:,:] = sample[0]
            
            target_q[i, sample[2]] = sample[3]
            if not game_over_:
                target_q[i, sample[2]] += self.discount*np.max(self.model.predict(sample[1].reshape(1,5,5,self.n_state)).flatten())
        
        target_q = np.clip(target_q, -30, 30)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model


def my_custom_loss(y_true,y_pred):
    """
    Let's have a look at the issue when using the MSE loss. Take for example the situation in which :
       - Target = [   0, 0.8,    0,   0]
       - Output = [-0.3, 0.1, -1.5, 4.1]
    Then the MSE loss will also consider the distance between 0 and the conrresponding output value even though
    0 means that we don't have the information since the action wasn't considered. Thus the algorithm is mislead
    with a wrong information and it actually learns either to reproduce it's output, or to only output values
    close to 0.
    
    What could be much more meaningful would be to only consider the squared deviation between the target and
    the output but only for the information that we have.
    """
    norm = K.sum(y_true, axis=-1)+1e-6 # Get the value of the non-zero element from the target
    norm = K.reshape(norm, shape=(-1,1)) # Reshape it in order to concatenate it after
    norm = K.concatenate([norm, norm, norm, norm],axis=-1) # Concatenate in order to allow the next operation
    loss = K.sum(K.square(y_true * (y_true - y_pred) / norm), axis=-1) # Multiply the deviation by the target
    # thus the non selected actions won't be considered in the loss (and divide by the norm of the target not 
    # to impact the final loss)
    return loss

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten())
        model.add(Dense(32))
        model.add(Dense(64))
        model.add(Dense(4))
        
        loss = my_custom_loss #"mse"
        optimizer = Adam(lr=lr) #sgd(lr=lr, decay=1e-4, momentum=0.0), loss="mse")
        
        model.compile(optimizer=optimizer, loss=loss)
        self.model = model
        

In [97]:
K.clear_session()
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.0001, epsilon = 0.3, memory_size=2000, batch_size = 64)
train(agent, env, epoch=20, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

# we get impressively good results ! 
# It could be interesting to select the best set of parameters with a grid search

Epoch 001/020 | Loss 0.0000 | Win/lose count 1.5/3.0 (-1.5)
Epoch 002/020 | Loss 0.0228 | Win/lose count 6.0/7.0 (-1.0)
Epoch 003/020 | Loss 0.0760 | Win/lose count 5.0/14.0 (-9.0)
Epoch 004/020 | Loss 0.0027 | Win/lose count 8.0/6.0 (2.0)
Epoch 005/020 | Loss 0.0209 | Win/lose count 9.0/4.0 (5.0)
Epoch 006/020 | Loss 0.0184 | Win/lose count 6.5/7.0 (-0.5)
Epoch 007/020 | Loss 0.0010 | Win/lose count 6.5/5.0 (1.5)
Epoch 008/020 | Loss 0.0000 | Win/lose count 7.0/2.0 (5.0)
Epoch 009/020 | Loss 0.0264 | Win/lose count 11.0/10.0 (1.0)
Epoch 010/020 | Loss 0.0000 | Win/lose count 5.0/5.0 (0.0)
Epoch 011/020 | Loss 0.0000 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/020 | Loss 0.0149 | Win/lose count 15.0/4.0 (11.0)
Epoch 013/020 | Loss 0.0067 | Win/lose count 13.5/5.0 (8.5)
Epoch 014/020 | Loss 0.0035 | Win/lose count 9.5/3.0 (6.5)
Epoch 015/020 | Loss 0.0448 | Win/lose count 16.0/4.0 (12.0)
Epoch 016/020 | Loss 0.0535 | Win/lose count 10.0/4.0 (6.0)
Epoch 017/020 | Loss 0.0141 | Win/lose coun

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [85]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(16))
        model.add(Dense(4))
        
        loss = my_custom_loss #"mse"
        optimizer = Adam(lr=lr) #sgd(lr=lr, decay=1e-4, momentum=0.0), loss="mse")
        
        model.compile(optimizer=optimizer, loss=loss)
        self.model = model

In [88]:
K.clear_session()
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.2, memory_size=2000, batch_size = 64)
train(agent,env,epoch=20,prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

### Comments :
# The loss represents the learning of the algorithm and the win/lose count represents somehow how well does the
# algorithm generalizes.

Epoch 001/020 | Loss 0.0018 | Win/lose count 5.0/6.0 (-1.0)
Epoch 002/020 | Loss 0.0015 | Win/lose count 7.0/4.0 (3.0)
Epoch 003/020 | Loss 0.1199 | Win/lose count 7.5/6.0 (1.5)
Epoch 004/020 | Loss 0.0089 | Win/lose count 16.0/4.0 (12.0)
Epoch 005/020 | Loss 0.0128 | Win/lose count 11.5/6.0 (5.5)
Epoch 006/020 | Loss 0.0267 | Win/lose count 13.0/5.0 (8.0)
Epoch 007/020 | Loss 0.0519 | Win/lose count 17.5/6.0 (11.5)
Epoch 008/020 | Loss 0.0042 | Win/lose count 14.0/6.0 (8.0)
Epoch 009/020 | Loss 0.0157 | Win/lose count 10.0/4.0 (6.0)
Epoch 010/020 | Loss 0.0353 | Win/lose count 17.5/6.0 (11.5)
Epoch 011/020 | Loss 0.0000 | Win/lose count 11.5/3.0 (8.5)
Epoch 012/020 | Loss 0.0000 | Win/lose count 6.0/2.0 (4.0)
Epoch 013/020 | Loss 0.0122 | Win/lose count 17.5/8.0 (9.5)
Epoch 014/020 | Loss 0.0048 | Win/lose count 9.5/3.0 (6.5)
Epoch 015/020 | Loss 0.0223 | Win/lose count 20.0/9.0 (11.0)
Epoch 016/020 | Loss 0.0173 | Win/lose count 13.5/5.0 (8.5)
Epoch 017/020 | Loss 0.0507 | Win/lose c

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [89]:
K.clear_session()
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 10.0/2.0. Average score (8.0)
Win/lose count 17.0/6.0. Average score (9.5)
Win/lose count 18.0/4.0. Average score (11.0)
Win/lose count 12.5/2.0. Average score (10.875)
Win/lose count 16.0/1.0. Average score (11.7)
Win/lose count 18.0/6.0. Average score (11.75)
Win/lose count 23.0/2.0. Average score (13.071428571428571)
Win/lose count 15.0/2.0. Average score (13.0625)
Win/lose count 8.5/4.0. Average score (12.11111111111111)
Win/lose count 16.5/4.0. Average score (12.15)
Final score: 12.15
Test of the FC
Win/lose count 4.5/2.0. Average score (2.5)
Win/lose count 6.5/0. Average score (4.5)
Win/lose count 4.5/1.0. Average score (4.166666666666667)
Win/lose count 6.0/2.0. Average score (4.125)
Win/lose count 3.0/0. Average score (3.9)
Win/lose count 7.0/1.0. Average score (4.25)
Win/lose count 4.0/0. Average score (4.214285714285714)
Win/lose count 17.0/2.0. Average score (5.5625)
Win/lose count 7.5/2.0. Average score (5.555555555555555)
Win/lose count 13.5/

In [90]:
HTML(display_videos('cnn_test10.mp4'))

In [76]:
HTML(display_videos('fc_test10.mp4'))

> As one might have foreseen, the CNN model performs better than the Fully connected one. Indeed, convolutions get local and spatial information with relations between states and thus it can get somehow the possible reward of a region rather than the reward of a single location. It stands to reason that this works much better ! It could be interesting to increase the dimensions of the board and give the mouse a wider vision of its local environment. With such a setting, the CNN should highly probably work much better than the FC and thus the differences would be much more obvious.

> The local loss I introduced overcomes the issues related to the MSE one. In fact this is questionable since the MSE tends to make the network reproduce what it has done and penalizes bad moves (0 is better than negative rewards), but in our case, if we give a good exploration parameter we authorize the model to perform badly at first in order to get some cases to be able to predict future rewards. Once it has learned how to predict future rewards, it is thought to behave well but then would it get that bad rewards are worse than 0 ? This is questionable. In our case at least, it seems to work impressively well in very few epochs !!

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [91]:
def train_explore(agent,env,epoch,eps_decay=0.5,eps_step=3,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    eps = agent.epsilon

    for e in range(epoch):
        if e!=0 and e%eps_step==0:
            eps = eps*eps_decay
        agent.set_epsilon(eps)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            # We modify this interval because we only care about the true reward 
            # and not the penalty of coming back to visited places. This penalty aims at helping the algorithm
            # itself but shouldn't be considered in the win/lose counts
            if reward > 0:
                win = win + reward
            if reward < -0.1:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if (e+1) % 10 == 0:
            env.draw(prefix+'_explore_'+str(e+1))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e+1, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'explore_model.h5',name_model=prefix+'_exploremodel.json')
        
        
class EnvironmentExploring(Environment):
    def __init__(self, *args, grid_size=13, max_time=200, temperature=0.3, **kwargs):
        super(EnvironmentExploring, self).__init__(*args,grid_size=grid_size, max_time=max_time, 
                                                   temperature=temperature,**kwargs)
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[:2, :] = -1
        self.position[:, :2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward += self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = -.1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over
    
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        bonus = 0.5*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2, :] = -1
        self.position[:, :2] = -1
        self.position[-2:,:] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0 # since we assume that we don't consider the first state in the reward,
        self.malus_position[self.x, self.y] = -.1 # we can consider the original position as visited (or not ?)
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [93]:
# Training
K.clear_session()
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon=0.3, memory_size=2000, batch_size=32, n_state=3)
train_explore(agent, env, epoch=20, prefix='cnn_train')
HTML(display_videos('cnn_train_explore_20.mp4'))

Epoch 001/020 | Loss 0.0333 | Win/lose count 8.5/8.0 (0.5)
Epoch 002/020 | Loss 0.2005 | Win/lose count 18.0/6.0 (12.0)
Epoch 003/020 | Loss 0.1003 | Win/lose count 16.0/4.0 (12.0)
Epoch 004/020 | Loss 0.2568 | Win/lose count 13.0/1.0 (12.0)
Epoch 005/020 | Loss 0.9159 | Win/lose count 19.0/6.0 (13.0)
Epoch 006/020 | Loss 0.0924 | Win/lose count 18.5/7.0 (11.5)
Epoch 007/020 | Loss 0.2712 | Win/lose count 23.0/5.0 (18.0)
Epoch 008/020 | Loss 0.0182 | Win/lose count 19.0/0 (19.0)
Epoch 009/020 | Loss 0.0202 | Win/lose count 19.5/4.0 (15.5)
Epoch 010/020 | Loss 0.0732 | Win/lose count 13.5/5.0 (8.5)
Epoch 011/020 | Loss 0.1472 | Win/lose count 22.5/3.0 (19.5)
Epoch 012/020 | Loss 0.0360 | Win/lose count 22.0/4.0 (18.0)
Epoch 013/020 | Loss 0.0299 | Win/lose count 22.0/2.0 (20.0)
Epoch 014/020 | Loss 0.0588 | Win/lose count 18.5/1.0 (17.5)
Epoch 015/020 | Loss 0.0249 | Win/lose count 23.0/3.0 (20.0)
Epoch 016/020 | Loss 0.0987 | Win/lose count 16.5/2.0 (14.5)
Epoch 017/020 | Loss 0.1005 |

In [96]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 19.5/3.0. Average score (16.5)
Win/lose count 14.5/2.0. Average score (14.5)
Win/lose count 23.0/2.0. Average score (16.666666666666668)
Win/lose count 11.0/2.0. Average score (14.75)
Win/lose count 20.0/4.0. Average score (15.0)
Win/lose count 17.5/2.0. Average score (15.083333333333334)
Win/lose count 13.0/0. Average score (14.785714285714286)
Win/lose count 17.0/1.0. Average score (14.9375)
Win/lose count 15.0/0. Average score (14.944444444444445)
Win/lose count 18.0/2.0. Average score (15.05)
Final score: 15.05


> This approach works impressively well ! Indeed there is a local minimum in our original approach which needed to be avoided : since the model itself tries to be confident about its prediction somehow, one can think of the situation in which the mouse goes back and forth on 2 possible positions. Indeed, it will predict that the expected future reward is 0, which is true since it doesn't explore anymore and thus with a current reward of 0 and a future reward of 0, the model is able to predict very precisely its future and thus is in a local minimum. We need to avoid this situation by forcing an exploration. We need to put a high exploration parameter at first and then decrease it when the model has enough data to train on. The other part of the new approach deals with giving a minus reward when coming back on visited locations, which is great since the model is not able to predict the future reward precisely (infinite instead of 0).

> This approach yields impressively good results ! However, since we have added some bias to our network with additional information, this approach might not generalize well to other games and thus, we will try in the next question to train a network that mimics the evolution of this network. Such a model should then generalize better and hopefully trully get the purpose of the game...

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [126]:
class Mimic(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(Mimic, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        pred_rwd_per_action = self.model.predict(s.reshape([1,5,5,self.n_state]))
        return np.argmax(pred_rwd_per_action)

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_])
        # Now we will not use the fact that our model need to verify the bellman equation 
        # but directly copy what the expert has done.
        # Hence we juste need the current state and the action that has been applied.
        input_states = np.zeros((self.batch_size,5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            state, action = self.memory.random_access()
            input_states[i] = state
            target_q[i,action] = 1 
            # Now we want to mimick the expert hence we want our target function to be maximized 
            # in the action applied by the expert. Thus we create a one hot vector based on the expert's decision.
            
        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class CNN_Mimic(Mimic):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(CNN_Mimic, self).__init__(*args,**kwargs)
        
        ### Mimic model constitution
        # Since the approach is somehow related to a teacher student approach which is usually used to
        # compress models, we will use a smaller network, the 'child'.
        
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
        model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
        model.add(Flatten())
        model.add(Dense(16))
        model.add(Dense(4, activation='softmax'))
        
        loss = "categorical_crossentropy"
        optimizer = Adam(lr=lr) #sgd(lr=lr, decay=1e-4, momentum=0.0), loss="mse")
        
        model.compile(optimizer=optimizer, loss=loss)
        self.model = model

In [143]:
def train_mimic(expert,agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0    
    
    for e in range(1, epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = expert.act(state,train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win += reward
            if reward < -0.1:
                lose -= reward

            # Train the agent and get the training loss
            loss = agent.reinforce(prev_state,  action)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [160]:
expert = DQN_CNN(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32, n_state=3)
expert.load(name_weights='cnn_trainexplore_model.h5',name_model='cnn_train_exploremodel.json')
mimic_agent = CNN_Mimic(size, lr=.0001, epsilon=0.1, memory_size=2000, batch_size=64, n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=500, temperature=0.3)
train_mimic(expert, mimic_agent, env, epoch=20, prefix='cnn_train_mimic')

Epoch 001/020 | Loss 0.0150 | Win/lose count 7.0/1.0 (6.0)
Epoch 002/020 | Loss 0.0062 | Win/lose count 8.5/1.0 (7.5)
Epoch 003/020 | Loss 0.0014 | Win/lose count 24.5/4.0 (20.5)
Epoch 004/020 | Loss 0.0003 | Win/lose count 13.0/0 (13.0)
Epoch 005/020 | Loss 0.0041 | Win/lose count 20.5/2.0 (18.5)
Epoch 006/020 | Loss 0.0007 | Win/lose count 17.0/1.0 (16.0)
Epoch 007/020 | Loss 0.0004 | Win/lose count 14.0/2.0 (12.0)
Epoch 008/020 | Loss 0.0004 | Win/lose count 8.0/1.0 (7.0)
Epoch 009/020 | Loss 0.0006 | Win/lose count 22.0/1.0 (21.0)
Epoch 010/020 | Loss 0.0024 | Win/lose count 14.5/0 (14.5)
Epoch 011/020 | Loss 0.0006 | Win/lose count 19.5/2.0 (17.5)
Epoch 012/020 | Loss 0.0013 | Win/lose count 17.0/1.0 (16.0)
Epoch 013/020 | Loss 0.0005 | Win/lose count 15.5/0 (15.5)
Epoch 014/020 | Loss 0.0004 | Win/lose count 10.0/1.0 (9.0)
Epoch 015/020 | Loss 0.0002 | Win/lose count 20.0/1.0 (19.0)
Epoch 016/020 | Loss 0.0022 | Win/lose count 17.5/1.0 (16.5)
Epoch 017/020 | Loss 0.0006 | Win/los

In [161]:
def test_mimic(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(1,epochs+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            state = state[:,:,1:]
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < -0.1:
                lose = lose - reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/e))
    print('Final score: '+str(score/epochs))

In [164]:
env_test = EnvironmentExploring(grid_size=size, max_time=500, temperature=0.3)
test_mimic(mimic_agent, env_test, epochs_test, prefix='cnn_test_mimic')
HTML(display_videos('cnn_test_mimic10.mp4'))

> As we can see our algorithm that tries to mimick the expert performs similarly to the algorithm without exploration ! 

> The final score for the mimic is lower than the score obtained using the approach with exploration. This can be explained by the fact that it doesn't really learn to improve the score but it just tries to copy the original algorithm hence it could at best perform as well as the expert algorithm but it couldn't exceed it.

***